In [2]:
! pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-n_tijlu6
  Running command git clone --filter=blob:none -q https://github.com/openai/CLIP.git /tmp/pip-req-build-n_tijlu6
  Resolved https://github.com/openai/CLIP.git to commit d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 64 kB 4.4 MB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369396 sha256=9a70c3da4439d00f618386877849a7e5b6be17cd6eaec75d2640cb5ec2dcc199
  Stored in directory: /tmp/pip-ephem-wheel-cache-j8xp2a7h/wheels/41/36/e0/af6191bd08c254b17d779b05de8282c9ea81ef7048abbe290b
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41913 sha256=4361761966c5e0048d871e3a4efcece0354df4192c368365e9218f8d0766da18
  Stored in directory: /home/pamessina/.cache/pip/wheels/ff/2a/24/75041425faf3347ab146a4a3d0484f723b2c44a7966a06e3f0
Successfully built clip ftfy


In [3]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [5]:
import torch
import numpy as np
from PIL import Image

In [7]:
import clip

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [52]:
model, preprocess = clip.load("RN50")
model.to(device).eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Número de parámetros:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Resolución de Entrada:", input_resolution)
print("Tamaño del contexto:", context_length)
print(f"Tamaño del vocabulario: {vocab_size:,}")

100%|███████████████████████████████████████| 244M/244M [01:17<00:00, 3.30MiB/s]


Número de parámetros: 102,007,137
Resolución de Entrada: 224
Tamaño del contexto: 77
Tamaño del vocabulario: 49,408


In [53]:
original_image = Image.open('/mnt/workspace/vinbig-cxr/dataset-png/images-256x256/6db32897e4b479fcf553e489b4cd255f.png')
image = preprocess(original_image)
image_input = image.unsqueeze(0).to(device)

In [54]:
image_input.shape

torch.Size([1, 3, 224, 224])

In [55]:
preprocess

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x7fbd71321f28>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [56]:
with torch.no_grad():
    image_features = model.encode_image(image_input)
#     image_features = model.encode_image(image_input).float()
#     image_features /= image_features.norm(dim=-1, keepdim=True)    

In [58]:
image_features.shape

torch.Size([1, 1024])

In [61]:
def custom_forward(self, x: torch.Tensor, return_local_features=False):
    x = self.conv1(x)  # shape = [*, width, grid, grid]
    x = x.reshape(x.shape[0], x.shape[1], -1)  # shape = [*, width, grid ** 2]
    x = x.permute(0, 2, 1)  # shape = [*, grid ** 2, width]
    x = torch.cat([self.class_embedding.to(x.dtype) + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device), x], dim=1)  # shape = [*, grid ** 2 + 1, width]
    x = x + self.positional_embedding.to(x.dtype)
    x = self.ln_pre(x)

    x = x.permute(1, 0, 2)  # NLD -> LND
    x = self.transformer(x)
    x = x.permute(1, 0, 2)  # LND -> NLD
    
    if return_local_features:

        global_features = self.ln_post(x[:, 0, :])
        
        print(global_features.shape)
        
        if self.proj is not None:
            global_features = global_features @ self.proj
            
        print(global_features.shape)

        local_features = x[:, 1:, :]

        return global_features, local_features

    else:
        x = self.ln_post(x[:, 0, :])

        if self.proj is not None:
            x = x @ self.proj
        
        return x
    
def resnet_custom_forward(self, x):
    def stem(x):
        x = self.relu1(self.bn1(self.conv1(x)))
        x = self.relu2(self.bn2(self.conv2(x)))
        x = self.relu3(self.bn3(self.conv3(x)))
        x = self.avgpool(x)
        return x

    x = x.type(self.conv1.weight.dtype)
    x = stem(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    pooled_x = self.attnpool(x)

    return x, pooled_x

In [62]:
model.visual.forward = resnet_custom_forward.__get__(model.visual)

In [63]:
tmp = model.visual(image_input.type(model.dtype))

In [70]:
tmp[0].shape

torch.Size([1, 2048, 7, 7])

In [32]:
model.visual.

OrderedDict([('class_embedding',
              tensor([ 2.0591e-03,  7.9107e-04, -7.4747e-03, -7.6637e-03, -6.3171e-03,
                      -2.6693e-03,  1.2999e-02, -5.1659e-03,  6.9023e-01, -7.2976e-03,
                      -1.3655e-01, -5.9287e-03,  5.6975e-03, -2.6576e-02, -4.3486e-03,
                      -4.8830e-03,  2.2827e-02, -4.2716e-03, -1.9288e-02, -4.0855e-04,
                       1.0054e-02, -1.1419e-03,  7.7314e-03,  4.0492e-03, -6.9799e-04,
                      -5.7392e-03,  8.7129e-02,  1.7433e-02, -9.4172e-03, -9.6271e-04,
                       6.1759e-04, -1.4820e-02, -1.0836e-01,  1.5677e-01, -1.3539e-02,
                       1.8349e-02,  8.0158e-03, -4.5944e-03, -4.4241e-03, -2.3373e-03,
                      -4.8460e-03,  6.7645e-03, -2.5592e-03, -3.6439e-01,  3.3991e-04,
                       4.5819e-03, -1.2367e-02, -2.2806e-02,  3.0317e-02, -1.3982e-02,
                      -1.7476e-03, -5.6374e-03, -6.2420e-03, -1.2713e-02,  7.6120e-01,
          

In [31]:
image_features.shape

torch.Size([1, 512])

In [26]:
image_features.shape

torch.Size([1, 512])

In [22]:
image_input

tensor([[[[-1.6609, -1.6755, -1.6755,  ..., -1.0039, -1.6609, -1.7631],
          [-1.6755, -1.6755, -1.6755,  ..., -0.9893, -1.5733, -1.7923],
          [-1.6755, -1.6755, -1.6901,  ..., -0.9310, -1.6025, -1.7631],
          ...,
          [ 0.1931,  0.2953,  0.4559,  ..., -1.7339, -1.7339, -1.7339],
          [ 0.1785,  0.2661,  0.4267,  ..., -1.7339, -1.7339, -1.7339],
          [ 0.0325,  0.1201,  0.2369,  ..., -1.7339, -1.7339, -1.7339]],

         [[-1.6170, -1.6320, -1.6320,  ..., -0.9417, -1.6170, -1.7221],
          [-1.6320, -1.6320, -1.6320,  ..., -0.9267, -1.5270, -1.7521],
          [-1.6320, -1.6320, -1.6470,  ..., -0.8666, -1.5570, -1.7221],
          ...,
          [ 0.2890,  0.3940,  0.5591,  ..., -1.6921, -1.6921, -1.6921],
          [ 0.2740,  0.3640,  0.5291,  ..., -1.6921, -1.6921, -1.6921],
          [ 0.1239,  0.2139,  0.3340,  ..., -1.6921, -1.6921, -1.6921]],

         [[-1.3522, -1.3665, -1.3665,  ..., -0.7123, -1.3522, -1.4518],
          [-1.3665, -1.3665, -

In [14]:
model.visual()

VisionTransformer(
  (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
  (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (transformer): Transformer(
    (resblocks): Sequential(
      (0): ResidualAttentionBlock(
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): QuickGELU()
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
      (1): ResidualAttentionBlock(
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise

In [2]:
import torch

In [3]:
tmp = torch.load('/home/pamessina/pretrained_weights/PubMedCLIP_RN50.pth')

In [7]:
tmp.keys()

dict_keys(['best_epoch', 'state_dict', 'optimizer_state_dict', 'best_loss'])

In [13]:
tmp['best_loss']

1.2550072763480393

In [22]:
from transformers import AutoProcessor, AutoModel

processor = AutoProcessor.from_pretrained("CenIA/vte-vit-base-patch16-bio-clinical-bert-finetuned", use_auth_token=True)

model = AutoModel.from_pretrained("CenIA/vte-vit-base-patch16-bio-clinical-bert-finetuned", use_auth_token=True)

In [23]:
from PIL import Image

In [24]:
original_image = Image.open('/mnt/workspace/vinbig-cxr/dataset-png/images-256x256/6db32897e4b479fcf553e489b4cd255f.png')

In [25]:
from medvqa.datasets.image_processing import get_image_transform

In [26]:
t = get_image_transform(image_size=(224, 224), use_clip_transform=True)

Returning default transform


In [27]:
# t(original_image.convert('RGB')).unsqueeze(0).shape
t(original_image.convert('RGB'))

tensor([[[-1.6609, -1.6755, -1.6755,  ..., -1.0039, -1.6609, -1.7631],
         [-1.6755, -1.6755, -1.6755,  ..., -0.9893, -1.5733, -1.7923],
         [-1.6755, -1.6755, -1.6901,  ..., -0.9310, -1.6025, -1.7631],
         ...,
         [ 0.1931,  0.2953,  0.4559,  ..., -1.7339, -1.7339, -1.7339],
         [ 0.1785,  0.2661,  0.4267,  ..., -1.7339, -1.7339, -1.7339],
         [ 0.0325,  0.1201,  0.2369,  ..., -1.7339, -1.7339, -1.7339]],

        [[-1.6170, -1.6320, -1.6320,  ..., -0.9417, -1.6170, -1.7221],
         [-1.6320, -1.6320, -1.6320,  ..., -0.9267, -1.5270, -1.7521],
         [-1.6320, -1.6320, -1.6470,  ..., -0.8666, -1.5570, -1.7221],
         ...,
         [ 0.2890,  0.3940,  0.5591,  ..., -1.6921, -1.6921, -1.6921],
         [ 0.2740,  0.3640,  0.5291,  ..., -1.6921, -1.6921, -1.6921],
         [ 0.1239,  0.2139,  0.3340,  ..., -1.6921, -1.6921, -1.6921]],

        [[-1.3522, -1.3665, -1.3665,  ..., -0.7123, -1.3522, -1.4518],
         [-1.3665, -1.3665, -1.3665,  ..., -0

In [28]:
processor

VisionTextDualEncoderProcessor:
- feature_extractor: ViTFeatureExtractor {
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "processor_class": "VisionTextDualEncoderProcessor",
  "resample": 2,
  "size": 224
}

- tokenizer: PreTrainedTokenizerFast(name_or_path='CenIA/vte-vit-base-patch16-bio-clinical-bert-finetuned', vocab_size=28996, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [9]:
import inspect

In [14]:
model.vision_model

ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): PatchEmbeddings(
      (projection): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0): ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=1024, out_features=4096, bias=True)
          (intermediate_act_fn): GELUActivation()

In [18]:
# print(inspect.getsource(model.vision_model.vision_model.forward))

In [29]:
tmp = model.vision_model(t(original_image.convert('RGB')).unsqueeze(0))

In [30]:
tmp.pooler_output.shape

torch.Size([1, 768])

In [31]:
tmp.last_hidden_state.shape

torch.Size([1, 197, 768])

In [17]:
model.vision_model.float()

ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): PatchEmbeddings(
      (projection): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0): ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=1024, out_features=4096, bias=True)
          (intermediate_act_fn): GELUActivation()

In [8]:
x = processor.feature_extractor(original_image.convert('RGB'))['pixel_values'][0]

In [11]:
x

array([[[-0.92941177, -0.9372549 , -0.9372549 , ..., -0.5372549 ,
         -0.8980392 , -0.9764706 ],
        [-0.9372549 , -0.9372549 , -0.9372549 , ..., -0.5137255 ,
         -0.8509804 , -0.9764706 ],
        [-0.9372549 , -0.9372549 , -0.94509804, ..., -0.52156866,
         -0.8509804 , -0.96862745],
        ...,
        [ 0.07450986,  0.12941182,  0.20784318, ..., -0.96862745,
         -0.96862745, -0.96862745],
        [ 0.06666672,  0.11372554,  0.19215691, ..., -0.96862745,
         -0.96862745, -0.96862745],
        [-0.00392157,  0.04313731,  0.09803927, ..., -0.96862745,
         -0.96862745, -0.96862745]],

       [[-0.92941177, -0.9372549 , -0.9372549 , ..., -0.5372549 ,
         -0.8980392 , -0.9764706 ],
        [-0.9372549 , -0.9372549 , -0.9372549 , ..., -0.5137255 ,
         -0.8509804 , -0.9764706 ],
        [-0.9372549 , -0.9372549 , -0.94509804, ..., -0.52156866,
         -0.8509804 , -0.96862745],
        ...,
        [ 0.07450986,  0.12941182,  0.20784318, ..., -